#**Imports and Util**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'AI Capstone/Colab Notebooks/Data Filtering'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#This was all copied from homework colabs from last semester

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Was used for a test requirenemt. I might readd in the future, but right now it is not needed.
#!pip install sdv

In [3]:
import requests
import pandas as pd
from io import BytesIO

import torch.nn as nn
import torch as T
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from collections import OrderedDict

from filtering_model import FilteringModel

In [4]:
#Credit to https://huggingface.co/thenlper/gte-base
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [5]:
def loadData(link):
  req = requests.get(link)
  content = BytesIO(req.content)
  df = pd.read_csv(content)
  return df

#**Dataset Loading and Analysis**

In [6]:
food_data = loadData("https://raw.githubusercontent.com/ezramuskat/Ingredient-Substitution-Capstone/main/data_preparation/classification_dataset/common_ingredients.csv")
food_data

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,olive oil,yes,yes,yes,yes
2,onions,yes,yes,yes,yes
3,water,yes,yes,yes,yes
4,garlic,yes,yes,yes,yes
...,...,...,...,...,...
494,boneless chicken breast,no,no,yes,yes
495,crème fraîche,yes,no,no,yes
496,cooked white rice,yes,yes,yes,yes
497,pecans,yes,yes,yes,yes


In [7]:
print((food_data[:] == 'yes').sum() / (food_data[:] != 'yes').sum())

ingredient      0.000000
vegetarian     11.794872
vegan           3.940594
dairy_free     10.880952
gluten_free    14.593750
dtype: float64


In [8]:
cheeseburger = ["salt","bread","meat","cheese","cheddar cheese","onions","chili","taco seasoned meat","worcestershire sauce","bug infested lettuce","cookie labeled eat me","lead pipe","your hopes and dreams"]

#**Filtering Model Class Experimentation**

##**facebook/drama-base**

In [9]:
model_name = "facebook/drama-base"

###**Regression Model**

In [10]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.599072098731995 | Val Loss: 2.569538652896881 | Train Acc: tensor(0.8440) | Val Acc: tensor(0.8925)
Epoch: 1 | Train Loss: 7.7530903816223145 | Val Loss: 2.311946153640747 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 2 | Train Loss: 6.998304575681686 | Val Loss: 1.9958301782608032 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 3 | Train Loss: 6.604564785957336 | Val Loss: 1.8470797836780548 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 4 | Train Loss: 5.933656334877014 | Val Loss: 1.7130012810230255 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 5 | Train Loss: 5.558473110198975 | Val Loss: 1.6032333076000214 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 6 | Train Loss: 5.466704100370407 | Val Loss: 1.4930552244186401 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.8925)
Epoch: 7 | Train Loss: 5.091489285230637 | Val Loss: 1.9455519914627075 | Train Acc: tensor(0.8929) | Val Acc: tensor(0

In [12]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,yes,yes,yes
2,meat,yes,no,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,yes,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,yes,yes,yes
8,worcestershire sauce,yes,no,yes,yes
9,bug infested lettuce,yes,yes,yes,yes


In [13]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [14]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [15]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.740114569664001 | Val Loss: 2.6394243240356445 | Train Acc: tensor(0.6153) | Val Acc: tensor(0.8375)
Epoch: 1 | Train Loss: 7.457781136035919 | Val Loss: 2.545214354991913 | Train Acc: tensor(0.7293) | Val Acc: tensor(0.8275)
Epoch: 2 | Train Loss: 6.6438309252262115 | Val Loss: 2.411350667476654 | Train Acc: tensor(0.8102) | Val Acc: tensor(0.7600)
Epoch: 3 | Train Loss: 5.857987463474274 | Val Loss: 1.685625672340393 | Train Acc: tensor(0.8728) | Val Acc: tensor(0.8925)
Epoch: 4 | Train Loss: 5.2534370720386505 | Val Loss: 1.9635993242263794 | Train Acc: tensor(0.9154) | Val Acc: tensor(0.8500)
Epoch: 5 | Train Loss: 4.335515782237053 | Val Loss: 1.450862854719162 | Train Acc: tensor(0.9442) | Val Acc: tensor(0.9100)
Epoch: 6 | Train Loss: 3.5667033791542053 | Val Loss: 1.446627914905548 | Train Acc: tensor(0.9630) | Val Acc: tensor(0.9325)
Epoch: 7 | Train Loss: 2.9609155356884003 | Val Loss: 1.1255000978708267 | Train Acc: tensor(0.9781) | Val Acc: tensor(0

In [16]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,yes,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [17]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [18]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [19]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.489689707756042 | Val Loss: 2.796260893344879 | Train Acc: tensor(0.6429) | Val Acc: tensor(0.5275)
Epoch: 1 | Train Loss: 7.309853553771973 | Val Loss: 2.60600346326828 | Train Acc: tensor(0.7462) | Val Acc: tensor(0.6425)
Epoch: 2 | Train Loss: 6.679596692323685 | Val Loss: 2.154084265232086 | Train Acc: tensor(0.8008) | Val Acc: tensor(0.8725)
Epoch: 3 | Train Loss: 6.398911148309708 | Val Loss: 1.8862765431404114 | Train Acc: tensor(0.8440) | Val Acc: tensor(0.8300)
Epoch: 4 | Train Loss: 6.136935114860535 | Val Loss: 1.8426488935947418 | Train Acc: tensor(0.8703) | Val Acc: tensor(0.8500)
Epoch: 5 | Train Loss: 5.341120690107346 | Val Loss: 1.5735535323619843 | Train Acc: tensor(0.8929) | Val Acc: tensor(0.9025)
Epoch: 6 | Train Loss: 4.89733549952507 | Val Loss: 1.3765486776828766 | Train Acc: tensor(0.9117) | Val Acc: tensor(0.9200)
Epoch: 7 | Train Loss: 4.565590366721153 | Val Loss: 1.2333708256483078 | Train Acc: tensor(0.9236) | Val Acc: tensor(0.917

In [20]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,yes,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [21]:
del fmodel_internal_model
del fmodel

##**sentence-transformers/all-MiniLM-L6-v2**

In [9]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

[autoreload of filtering_model failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/imp.py", line 315, in reload
    return importlib.reload(module)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/

###**Regression Model**

In [10]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.783885657787323 | Val Loss: 2.6042430996894836 | Train Acc: tensor(0.7907) | Val Acc: tensor(0.9125)
Epoch: 1 | Train Loss: 8.273582577705383 | Val Loss: 2.462473511695862 | Train Acc: tensor(0.8878) | Val Acc: tensor(0.9150)
Epoch: 2 | Train Loss: 7.850416600704193 | Val Loss: 2.436404228210449 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 3 | Train Loss: 7.404351651668549 | Val Loss: 2.149151384830475 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 4 | Train Loss: 7.039493888616562 | Val Loss: 2.069499433040619 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 5 | Train Loss: 6.900767922401428 | Val Loss: 1.9639784693717957 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 6 | Train Loss: 6.446556359529495 | Val Loss: 1.8480805158615112 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 7 | Train Loss: 6.152272880077362 | Val Loss: 1.7492440044879913 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.91

In [12]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,yes,yes,yes
2,meat,yes,yes,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,no,yes,yes
8,worcestershire sauce,yes,yes,yes,yes
9,bug infested lettuce,no,yes,no,no


In [13]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [14]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [15]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.676974058151245 | Val Loss: 2.643861770629883 | Train Acc: tensor(0.6153) | Val Acc: tensor(0.7900)
Epoch: 1 | Train Loss: 7.412990599870682 | Val Loss: 2.3272470831871033 | Train Acc: tensor(0.7331) | Val Acc: tensor(0.8100)
Epoch: 2 | Train Loss: 6.457194566726685 | Val Loss: 1.9057189226150513 | Train Acc: tensor(0.7989) | Val Acc: tensor(0.8600)
Epoch: 3 | Train Loss: 5.960794538259506 | Val Loss: 2.477253258228302 | Train Acc: tensor(0.8596) | Val Acc: tensor(0.8550)
Epoch: 4 | Train Loss: 5.1256243288517 | Val Loss: 1.4583174586296082 | Train Acc: tensor(0.8991) | Val Acc: tensor(0.9050)
Epoch: 5 | Train Loss: 4.738561302423477 | Val Loss: 1.3687803149223328 | Train Acc: tensor(0.9317) | Val Acc: tensor(0.9200)
Epoch: 6 | Train Loss: 3.892368569970131 | Val Loss: 1.0890749096870422 | Train Acc: tensor(0.9580) | Val Acc: tensor(0.9600)
Epoch: 7 | Train Loss: 3.5095290690660477 | Val Loss: 1.0206629782915115 | Train Acc: tensor(0.9768) | Val Acc: tensor(0.9

In [16]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,no,no,no,no
9,bug infested lettuce,yes,yes,yes,yes


In [17]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [18]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(384, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [19]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.822879076004028 | Val Loss: 2.9160224199295044 | Train Acc: tensor(0.5890) | Val Acc: tensor(0.3200)
Epoch: 1 | Train Loss: 7.823271334171295 | Val Loss: 2.6374297738075256 | Train Acc: tensor(0.6930) | Val Acc: tensor(0.5700)
Epoch: 2 | Train Loss: 7.213333368301392 | Val Loss: 2.0675955414772034 | Train Acc: tensor(0.7644) | Val Acc: tensor(0.8350)
Epoch: 3 | Train Loss: 6.723369985818863 | Val Loss: 2.126261919736862 | Train Acc: tensor(0.8020) | Val Acc: tensor(0.7650)
Epoch: 4 | Train Loss: 6.127924352884293 | Val Loss: 1.9899762868881226 | Train Acc: tensor(0.8546) | Val Acc: tensor(0.8600)
Epoch: 5 | Train Loss: 5.92241358757019 | Val Loss: 1.8889984786510468 | Train Acc: tensor(0.8810) | Val Acc: tensor(0.8575)
Epoch: 6 | Train Loss: 5.2391840517520905 | Val Loss: 1.4773203134536743 | Train Acc: tensor(0.8979) | Val Acc: tensor(0.9350)
Epoch: 7 | Train Loss: 5.088149935007095 | Val Loss: 1.5403190851211548 | Train Acc: tensor(0.9223) | Val Acc: tensor(0

In [20]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [21]:
del fmodel_internal_model
del fmodel

##**nomic-ai/nomic-embed-text-v1**

In [9]:
model_name = "nomic-ai/nomic-embed-text-v1"

###**Regression Model**

In [10]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.649936020374298 | Val Loss: 2.5206799507141113 | Train Acc: tensor(0.8628) | Val Acc: tensor(0.8600)
Epoch: 1 | Train Loss: 7.875728666782379 | Val Loss: 2.4603349566459656 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 2 | Train Loss: 7.166326969861984 | Val Loss: 2.1497386395931244 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 3 | Train Loss: 6.674674034118652 | Val Loss: 1.9887369871139526 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 4 | Train Loss: 6.094844192266464 | Val Loss: 1.8554780781269073 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 5 | Train Loss: 5.693200320005417 | Val Loss: 1.7513887882232666 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 6 | Train Loss: 5.5774871706962585 | Val Loss: 1.6600781381130219 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.8600)
Epoch: 7 | Train Loss: 5.106540471315384 | Val Loss: 1.6003240644931793 | Train Acc: tensor(0.9010) | Val Acc: tensor

In [12]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,yes,yes,yes
2,meat,yes,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,no,yes,yes
8,worcestershire sauce,yes,no,yes,yes
9,bug infested lettuce,yes,yes,yes,yes


In [13]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [14]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [15]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.35698926448822 | Val Loss: 2.589175224304199 | Train Acc: tensor(0.6698) | Val Acc: tensor(0.8100)
Epoch: 1 | Train Loss: 6.583994626998901 | Val Loss: 2.1982865929603577 | Train Acc: tensor(0.7870) | Val Acc: tensor(0.9250)
Epoch: 2 | Train Loss: 5.848681688308716 | Val Loss: 1.6338774263858795 | Train Acc: tensor(0.8578) | Val Acc: tensor(0.9350)
Epoch: 3 | Train Loss: 5.121618241071701 | Val Loss: 1.6640104353427887 | Train Acc: tensor(0.9010) | Val Acc: tensor(0.9475)
Epoch: 4 | Train Loss: 4.527813658118248 | Val Loss: 1.1272588670253754 | Train Acc: tensor(0.9317) | Val Acc: tensor(0.9650)
Epoch: 5 | Train Loss: 3.841495007276535 | Val Loss: 1.2105295956134796 | Train Acc: tensor(0.9643) | Val Acc: tensor(0.9525)
Epoch: 6 | Train Loss: 3.4248463958501816 | Val Loss: 1.0540012419223785 | Train Acc: tensor(0.9637) | Val Acc: tensor(0.9575)
Epoch: 7 | Train Loss: 3.058008164167404 | Val Loss: 0.953030601143837 | Train Acc: tensor(0.9799) | Val Acc: tensor(0.

In [16]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [17]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [18]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [19]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.506073594093323 | Val Loss: 2.5762165188789368 | Train Acc: tensor(0.6748) | Val Acc: tensor(0.9100)
Epoch: 1 | Train Loss: 7.031326353549957 | Val Loss: 2.3522636890411377 | Train Acc: tensor(0.7682) | Val Acc: tensor(0.9300)
Epoch: 2 | Train Loss: 6.670316368341446 | Val Loss: 2.0115722715854645 | Train Acc: tensor(0.8145) | Val Acc: tensor(0.8850)
Epoch: 3 | Train Loss: 5.707792371511459 | Val Loss: 1.5226266086101532 | Train Acc: tensor(0.8778) | Val Acc: tensor(0.9375)
Epoch: 4 | Train Loss: 5.737126469612122 | Val Loss: 1.7081451416015625 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.8850)
Epoch: 5 | Train Loss: 5.415961265563965 | Val Loss: 1.6674744784832 | Train Acc: tensor(0.8997) | Val Acc: tensor(0.8600)
Epoch: 6 | Train Loss: 4.580310821533203 | Val Loss: 1.3881942629814148 | Train Acc: tensor(0.9330) | Val Acc: tensor(0.9200)
Epoch: 7 | Train Loss: 4.502478688955307 | Val Loss: 1.1778323948383331 | Train Acc: tensor(0.9361) | Val Acc: tensor(0.9

In [20]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [21]:
del fmodel_internal_model
del fmodel

##**jonny9f/food_embeddings2**

In [22]:
model_name = "jonny9f/food_embeddings2"

###**Regression Model**

In [23]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [24]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.818895280361176 | Val Loss: 2.590630531311035 | Train Acc: tensor(0.7732) | Val Acc: tensor(0.7925)
Epoch: 1 | Train Loss: 8.379445135593414 | Val Loss: 2.477961778640747 | Train Acc: tensor(0.8214) | Val Acc: tensor(0.8125)
Epoch: 2 | Train Loss: 8.093638241291046 | Val Loss: 2.3958566784858704 | Train Acc: tensor(0.8239) | Val Acc: tensor(0.8225)
Epoch: 3 | Train Loss: 7.783616364002228 | Val Loss: 2.1884503960609436 | Train Acc: tensor(0.8277) | Val Acc: tensor(0.8300)
Epoch: 4 | Train Loss: 7.521835744380951 | Val Loss: 2.371412992477417 | Train Acc: tensor(0.8333) | Val Acc: tensor(0.8300)
Epoch: 5 | Train Loss: 7.197302848100662 | Val Loss: 2.0773464739322662 | Train Acc: tensor(0.8390) | Val Acc: tensor(0.8300)
Epoch: 6 | Train Loss: 7.0723817348480225 | Val Loss: 1.9892677068710327 | Train Acc: tensor(0.8409) | Val Acc: tensor(0.8300)
Epoch: 7 | Train Loss: 6.78255170583725 | Val Loss: 2.015787124633789 | Train Acc: tensor(0.8477) | Val Acc: tensor(0.83

In [25]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,yes,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,yes,yes,yes
8,worcestershire sauce,yes,yes,yes,yes
9,bug infested lettuce,yes,yes,yes,yes


In [26]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [27]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [28]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.33203935623169 | Val Loss: 2.6784956455230713 | Train Acc: tensor(0.6867) | Val Acc: tensor(0.8600)
Epoch: 1 | Train Loss: 6.784210681915283 | Val Loss: 2.045914351940155 | Train Acc: tensor(0.8033) | Val Acc: tensor(0.9225)
Epoch: 2 | Train Loss: 6.434014141559601 | Val Loss: 1.9143327176570892 | Train Acc: tensor(0.8571) | Val Acc: tensor(0.8625)
Epoch: 3 | Train Loss: 5.906189650297165 | Val Loss: 2.0756121277809143 | Train Acc: tensor(0.8997) | Val Acc: tensor(0.8850)
Epoch: 4 | Train Loss: 5.251487880945206 | Val Loss: 1.8278631269931793 | Train Acc: tensor(0.9135) | Val Acc: tensor(0.9025)
Epoch: 5 | Train Loss: 4.792091518640518 | Val Loss: 1.535546988248825 | Train Acc: tensor(0.9373) | Val Acc: tensor(0.9100)
Epoch: 6 | Train Loss: 4.20213395357132 | Val Loss: 1.314245194196701 | Train Acc: tensor(0.9480) | Val Acc: tensor(0.9100)
Epoch: 7 | Train Loss: 3.4077690094709396 | Val Loss: 1.1955796629190445 | Train Acc: tensor(0.9612) | Val Acc: tensor(0.92

In [29]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,yes,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,yes,yes,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [30]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [31]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [32]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.328211069107056 | Val Loss: 2.8075246810913086 | Train Acc: tensor(0.7061) | Val Acc: tensor(0.3075)
Epoch: 1 | Train Loss: 7.383571267127991 | Val Loss: 2.594910144805908 | Train Acc: tensor(0.7932) | Val Acc: tensor(0.7125)
Epoch: 2 | Train Loss: 6.815333396196365 | Val Loss: 2.0531531274318695 | Train Acc: tensor(0.8509) | Val Acc: tensor(0.8550)
Epoch: 3 | Train Loss: 6.275689423084259 | Val Loss: 1.7987581193447113 | Train Acc: tensor(0.8665) | Val Acc: tensor(0.8475)
Epoch: 4 | Train Loss: 5.950261890888214 | Val Loss: 1.7702341377735138 | Train Acc: tensor(0.8910) | Val Acc: tensor(0.8975)
Epoch: 5 | Train Loss: 5.383463740348816 | Val Loss: 1.4187566637992859 | Train Acc: tensor(0.9254) | Val Acc: tensor(0.9225)
Epoch: 6 | Train Loss: 5.189377427101135 | Val Loss: 1.3921283185482025 | Train Acc: tensor(0.9286) | Val Acc: tensor(0.9150)
Epoch: 7 | Train Loss: 4.53625425696373 | Val Loss: 1.203397512435913 | Train Acc: tensor(0.9380) | Val Acc: tensor(0.9

In [33]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,yes,yes,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [34]:
del fmodel_internal_model
del fmodel

##**DivyaMereddy007/RecipeBert_v5**

In [35]:
model_name = "DivyaMereddy007/RecipeBert_v5"

###**Regression Model**

In [36]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [37]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.57479703426361 | Val Loss: 2.4996281266212463 | Train Acc: tensor(0.8628) | Val Acc: tensor(0.9150)
Epoch: 1 | Train Loss: 7.6826446652412415 | Val Loss: 2.2412227392196655 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 2 | Train Loss: 7.045786142349243 | Val Loss: 1.9964548349380493 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 3 | Train Loss: 6.409300148487091 | Val Loss: 2.079110473394394 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 4 | Train Loss: 5.966849148273468 | Val Loss: 2.016862839460373 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 5 | Train Loss: 5.552961647510529 | Val Loss: 1.4930157363414764 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 6 | Train Loss: 5.531769216060638 | Val Loss: 1.8551221191883087 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9150)
Epoch: 7 | Train Loss: 5.07651099562645 | Val Loss: 1.3390846252441406 | Train Acc: tensor(0.8872) | Val Acc: tensor(0.9

In [38]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,yes,yes,yes
2,meat,yes,no,yes,yes
3,cheese,yes,no,yes,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,yes,yes,yes
9,bug infested lettuce,yes,yes,yes,yes


In [39]:
del fmodel_internal_model
del fmodel

###**Small Model**

In [40]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 128)),
    ('dr1', nn.Dropout(0.25)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(128)),
    ('fc3', nn.Linear(128, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [41]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.064069986343384 | Val Loss: 2.6072765588760376 | Train Acc: tensor(0.6779) | Val Acc: tensor(0.8875)
Epoch: 1 | Train Loss: 6.76537823677063 | Val Loss: 2.1473177075386047 | Train Acc: tensor(0.7726) | Val Acc: tensor(0.8850)
Epoch: 2 | Train Loss: 6.104278326034546 | Val Loss: 1.9127067029476166 | Train Acc: tensor(0.8402) | Val Acc: tensor(0.8800)
Epoch: 3 | Train Loss: 5.550381720066071 | Val Loss: 1.7069266438484192 | Train Acc: tensor(0.8778) | Val Acc: tensor(0.9125)
Epoch: 4 | Train Loss: 5.036517262458801 | Val Loss: 1.6861083507537842 | Train Acc: tensor(0.9185) | Val Acc: tensor(0.9125)
Epoch: 5 | Train Loss: 4.281194895505905 | Val Loss: 1.8585049510002136 | Train Acc: tensor(0.9405) | Val Acc: tensor(0.9400)
Epoch: 6 | Train Loss: 3.850051775574684 | Val Loss: 4.009863972663879 | Train Acc: tensor(0.9662) | Val Acc: tensor(0.9350)
Epoch: 7 | Train Loss: 3.042399764060974 | Val Loss: 1.8785240948200226 | Train Acc: tensor(0.9668) | Val Acc: tensor(0.

In [42]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,yes,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [43]:
del fmodel_internal_model
del fmodel

###**Large Model**

In [44]:
fmodel_internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

fmodel = FilteringModel(food_data,"ingredient",model_name,fmodel_internal_model)


In [45]:
fmodel.train_model(epochs=20,batch_size=33,val_split=0.2)

Epoch: 0 | Train Loss: 8.879430413246155 | Val Loss: 3.016221284866333 | Train Acc: tensor(0.6009) | Val Acc: tensor(0.2525)
Epoch: 1 | Train Loss: 7.9240341782569885 | Val Loss: 2.7793562412261963 | Train Acc: tensor(0.6911) | Val Acc: tensor(0.4750)
Epoch: 2 | Train Loss: 7.317847460508347 | Val Loss: 2.388305962085724 | Train Acc: tensor(0.7381) | Val Acc: tensor(0.8100)
Epoch: 3 | Train Loss: 6.816078573465347 | Val Loss: 1.893489509820938 | Train Acc: tensor(0.7895) | Val Acc: tensor(0.8700)
Epoch: 4 | Train Loss: 6.628911465406418 | Val Loss: 1.773233324289322 | Train Acc: tensor(0.8239) | Val Acc: tensor(0.8800)
Epoch: 5 | Train Loss: 6.268487900495529 | Val Loss: 1.6965284049510956 | Train Acc: tensor(0.8634) | Val Acc: tensor(0.8925)
Epoch: 6 | Train Loss: 5.50951275229454 | Val Loss: 1.2310669422149658 | Train Acc: tensor(0.8878) | Val Acc: tensor(0.9450)
Epoch: 7 | Train Loss: 5.025468677282333 | Val Loss: 1.2241759896278381 | Train Acc: tensor(0.9167) | Val Acc: tensor(0.94

In [46]:
#Recipie for a non-kosher cheeseburger
fmodel.filter(cheeseburger)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,bread,yes,no,yes,no
2,meat,no,no,yes,yes
3,cheese,yes,no,no,yes
4,cheddar cheese,yes,no,no,yes
5,onions,yes,yes,yes,yes
6,chili,yes,no,yes,yes
7,taco seasoned meat,no,no,yes,yes
8,worcestershire sauce,yes,no,yes,no
9,bug infested lettuce,yes,yes,yes,yes


In [47]:
del fmodel_internal_model
del fmodel